<a href="https://colab.research.google.com/github/dkloepper/scooterbuddy/blob/main/ScooterBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q -U gmplot

     |████████████████████████████████| 164 kB 28.3 MB/s 


In [1]:
!pip install -q -U streamlit

     |████████████████████████████████| 9.2 MB 12.9 MB/s 
     |████████████████████████████████| 4.7 MB 78.1 MB/s 
     |████████████████████████████████| 182 kB 71.8 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 237 kB 67.5 MB/s 
     |████████████████████████████████| 164 kB 58.5 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 51 kB 6.7 MB/s 


In [2]:
!pip install -q -U folium
!pip install -q -U streamlit-folium

     |████████████████████████████████| 102 kB 6.6 MB/s 
     |████████████████████████████████| 422 kB 9.5 MB/s 


In [3]:
!pip install -q -U geopandas

     |████████████████████████████████| 1.1 MB 13.4 MB/s 
     |████████████████████████████████| 16.6 MB 54.6 MB/s 
     |████████████████████████████████| 7.8 MB 7.2 MB/s 


In [4]:
#Requires downgrade of SKlearn
#!pip install -U -q pycaret

In [5]:
import json
import numpy as np 
import pandas as pd
import os
import re

#import gmplot

from pandas.io.json import json_normalize
from matplotlib import pyplot as plt

from math import radians, cos, sin, asin, sqrt

from IPython.display import display
%matplotlib inline

In [6]:
from ipywidgets import embed
import streamlit.components.v1 as components
import streamlit as st

In [7]:
from streamlit_folium import st_folium
import folium

In [8]:
import geopandas as gpd

In [9]:
#from tqdm import tqdm
#from pycaret.regression import *

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

#import lightgbm as lgb
#from lightgbm import LGBMRegressor

In [11]:
import pickle

#Addresses Manipulation & Mapping

In [12]:
centerline_df = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/scooter_data/MPLS_Centerline.geojson')
address_df = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/scooter_data/EAS_Addresses.geojson')

In [13]:
#Drop rows where geometry is null
address_df.dropna(subset=['geometry'],inplace=True)

In [14]:
#Backup incase encoding needs to be restored
#centerline_df = centerline_df.to_crs(epsg=4326)
#address_df = address_df.to_crs(epsg=4326)

In [15]:
#Calculate centroids for centerlines
centerline_df['centroid'] = centerline_df.apply(lambda r: gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001).centroid, axis=1)

In [16]:
#Generate lat/long versions of centroids and address geos
centerline_df['latlon'] = centerline_df.apply(lambda r: "[{0},{1}]".format(r['centroid'].y, r['centroid'].x), axis=1)
address_df['latlon'] = address_df.apply(lambda r: "[{0},{1}]".format(r['geometry'].y, r['geometry'].x), axis=1)

In [17]:
#Convert measurements to encoding that measures in feet rather than radians
address_df = address_df.to_crs(epsg=6505)
centerline_df = centerline_df.to_crs(epsg=6505)

In [18]:
#Recalc centroids under new conversion
centerline_df['centroid'] = centerline_df.apply(lambda r: gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001).centroid, axis=1)

In [19]:
#address_df

In [20]:
#centerline_df

In [21]:
#Validating coordinates and latlog values
dest_coords = centerline_df.iloc[0]['centroid']
dest_latlong = centerline_df.iloc[0]['latlon']
origin_coords = address_df.iloc[0]['geometry']
origin_latlong = address_df.iloc[0]['latlon']

In [22]:
str(dest_coords)

'POINT (2824343.3433687184 1033437.1651246486)'

In [23]:
str(origin_coords)

'POINT (2818771.29028275 1050160.1539404693)'

In [24]:
#Test calculation in feed
foot_dist = origin_coords.distance(dest_coords)
foot_dist

17626.858214863827

In [25]:
#Convert to miles
miles_dist = foot_dist / 5279.98944
miles_dist

3.338426793305069

In [26]:
#address_df.crs

In [27]:
start = address_df.sample()
start

,EASAddressId,ObjectId,AddrNum,AddrNumSuffix,StreetNameID,StreetName,StreetType,PostDir,Unit,APN,...,IsAlias,AddressTypeDesc,LastUpdateDate,Display,EffectiveDate,InactiveDate,ZipCode,OBJECTID_1,geometry,latlon
63740,ADDR163742,69312,2701,None,487,FRANKLIN,AVE,E,None,3602924120012,...,N,Parcel,2018-05-21 13:23:11+00:00,2701 FRANKLIN AVE E,2012-05-11 15:06:36+00:00,NaT,55406,63741,POINT (2823287.875 1044424.850),"[44.9624787952659,-93.2326383128508]"


In [28]:
#start.iloc[0]['latlon']

In [29]:
origin_coords = start.iloc[0]['geometry']
origin_latlong = start.iloc[0]['latlon']

In [30]:
def find_nearest(coords,df):
    distances = df.apply(lambda r: origin_coords.distance(r['centroid']), 
        axis=1)
    return df.loc[distances.idxmin(), 'latlon']

In [31]:
def find_within_dist(coords,df,dist):
    df['distances'] = df.apply(lambda r: origin_coords.distance(r['centroid'])  / 5279.98944, 
        axis=1)
    return df[(df['distances'] <= dist)]

In [32]:
dest_latlong = find_nearest(origin_coords,centerline_df)
dest_latlong

'[44.9625045631921,-93.23299578299066]'

In [33]:
within_df = find_within_dist(origin_coords,centerline_df,0.1).sort_values(by=['distances'])
within_df

,OBJECTID,L_F_ADD,R_F_ADD,L_T_ADD,R_T_ADD,STREETALL,F_CLASS,ALT_NAM1,ALT_NAM2,ZIP5_L,...,T_XSTREET_2,T_XSTREET_3,GlobalID,FromLanes,ToLanes,Shape__Length,geometry,centroid,latlon,distances
12230,12231,2010,2011,2000,2001,27TH AVE S,A40,None,None,55406,...,None,None,7b589649-1658-4d05-9eed-35520e29ca06,1,1.0,81.516001,"LINESTRING (2823194.324 1044338.929, 2823196.2...",POINT (2823195.2632308723 1044433.3758917374),"[44.9625045631921,-93.23299578299066]",0.017614
7935,7936,2700,2701,2798,2799,FRANKLIN AVE E,A30,COUNTY ROAD 5,None,55406,...,None,None,251ef46b-42ad-4003-9fa3-1f7c84bd1bb6,1,1.0,176.805239,"LINESTRING (2823196.202 1044527.823, 2823607.2...",POINT (2823401.746954847 1044529.6453560749),"[44.96276328356035,-93.2321945980801]",0.029310
6067,6068,2600,2601,2698,2699,PAUTZ PL,A40,None,None,55406,...,None,None,4259bde5-3521-4bba-bbdb-2f6bf6e5069e,1,1.0,176.474129,"LINESTRING (2822784.002 1044335.198, 2823194.3...",POINT (2822989.163026198 1044337.0633494051),"[44.962245703771444,-93.2337954807746]",0.058967
1651,1652,2600,2601,2698,2699,FRANKLIN AVE E,A30,COUNTY ROAD 5,None,55406,...,None,None,8f346f5c-82d4-46af-96d0-34abf1e488eb,1,1.0,176.336863,"LINESTRING (2822786.211 1044523.237, 2823196.2...",POINT (2822991.206520281 1044525.53008107),"[44.962762562346896,-93.23378076139716]",0.059335
2757,2758,2198,2199,2012,2013,27TH AVE S,A40,None,None,55406,...,None,None,0d6f722c-192f-465a-a026-3c3616ad7781,1,1.0,200.636882,"LINESTRING (2823197.391 1043873.979, 2823194.3...",POINT (2823195.8576652366 1044106.4539616301),"[44.9616078931129,-93.23300533740095]",0.062770
8081,8082,998,999,900,901,27TH AVE S,A40,None,None,55406,...,None,None,f5e2aacb-fe12-4207-88bb-a593ae13c600,1,1.0,188.499919,"LINESTRING (2823193.189 1044964.633, 2823196.2...",POINT (2823194.6954828748 1044746.2279181016),"[44.96336264246505,-93.23298663525276]",0.063374
12474,12475,2198,2199,2000,2001,28TH AVE S,A40,None,None,55406,...,None,None,5cbc6718-5e5f-4434-85c6-fdee33356553,1,1.0,281.559946,"LINESTRING (2823605.962 1043878.980, 2823607.2...",POINT (2823606.6270620637 1044205.2235742622),"[44.96186820784435,-93.23141488264605]",0.073313
10726,10727,2899,2898,2713,2712,RIVERSIDE AVE,A40,None,None,55406,...,9TH ST S,None,30f48344-1e04-41a0-8bc6-78a202c609c1,1,1.0,369.325994,"LINESTRING (2824017.052 1044536.835, 2823976.9...",POINT (2823653.3130472233 1044762.6752261568),"[44.9633854287694,-93.23122523586636]",0.094255
1535,1536,2010,2011,2000,2001,26TH AVE S,A30,COUNTY ROAD 48,None,55406,...,None,None,e0d6d52b-753b-4589-a3e0-eaaf1dcee54e,1,1.0,81.148292,"LINESTRING (2822784.002 1044335.198, 2822786.2...",POINT (2822785.1063156063 1044429.2175387379),"[44.96250370292625,-93.2345804591811]",0.095225


In [34]:
m = folium.Map(location=json.loads(origin_latlong), zoom_start=18)
folium.Marker(json.loads(origin_latlong)).add_to(m)
folium.Marker(json.loads(dest_latlong),icon=folium.Icon(color='red')).add_to(m)
for _, r in within_df.iterrows():
  folium.Marker(json.loads(r['latlon']),icon=folium.Icon(color='green')).add_to(m)
#m

In [35]:
centerline_df['LOCATION'] = centerline_df.apply(lambda r: "{0} - {1} {2}".format(r['L_F_ADD'], r['R_T_ADD'], r['STREETALL']), axis=1)
centerline_df

,OBJECTID,L_F_ADD,R_F_ADD,L_T_ADD,R_T_ADD,STREETALL,F_CLASS,ALT_NAM1,ALT_NAM2,ZIP5_L,...,T_XSTREET_3,GlobalID,FromLanes,ToLanes,Shape__Length,geometry,centroid,latlon,distances,LOCATION
0,1,3000,3001,3098,3099,39TH ST E,A40,None,None,55406,...,None,29744cc9-5dea-4a48-a068-2daccb8ca02b,1,1.0,138.028160,"LINESTRING (2824182.848 1033436.019, 2824482.7...",POINT (2824343.3433687184 1033437.1651246486),"[44.932303725970755,-93.22896590095331]",2.090584,3000 - 3099 39TH ST E
1,2,601,600,613,612,SHERIDAN AVE N,A40,None,None,55411,...,None,bc64c65c-369f-49e9-9cfc-c46b5aedb84d,1,1.0,24.108303,"LINESTRING (2802664.075 1052261.529, 2802650.1...",POINT (2802651.7366334866 1052286.376582747),"[44.9845465414285,-93.31209854486539]",4.182373,601 - 612 SHERIDAN AVE N
2,3,1998,1999,1900,1901,57TH ST W,A40,None,None,55419,...,None,0b2b1962-c712-4854-8b5e-9430e8009df5,1,1.0,142.731382,"LINESTRING (2804792.395 1021375.391, 2805124.6...",POINT (2804958.5136064384 1021375.8432505609),"[44.8997109417065,-93.30420711579976]",5.577404,1998 - 1901 57TH ST W
3,4,600,601,618,619,40TH ST E,A40,None,None,55407,...,None,4de4c732-6273-4758-926a-fb5480518a57,1,1.0,140.356403,"LINESTRING (2814357.285 1032702.547, 2814683.8...",POINT (2814520.54720823 1032703.9520650019),"[44.930552156256596,-93.26691461978174]",2.772189,600 - 619 40TH ST E
4,5,3198,3199,3100,3101,46TH AVE S,A40,None,None,55406,...,None,89ca243a-d516-4e54-8a98-13bb4fa0d8ff,1,1.0,271.241016,"LINESTRING (2829825.169 1038146.805, 2829818.2...",POINT (2829821.695328156 1038461.1595069288),"[44.9459510802885,-93.207620164627]",1.675432,3198 - 3101 46TH AVE S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13833,13834,1200,1201,1298,1299,EAST MINNEHAHA PKWY,A40,MINNEHAHA PKWY E,None,55417,...,None,555a247a-b2ae-4167-9fbe-f29597ea52f0,1,1.0,148.079248,"LINESTRING (2817010.288 1026439.590, 2817095.3...",POINT (2817174.151779763 1026491.6806034332),"[44.9134285378557,-93.2568797935352]",3.588391,1200 - 1299 EAST MINNEHAHA PKWY
13834,13835,0,5401,0,5409,2ND AVE S,A40,None,None,55419,...,None,2a3e283c-2d82-4e40-ab67-5abd8a46d616,1,1.0,86.953259,"LINESTRING (2812708.453 1023238.598, 2812715.9...",POINT (2812712.3810812924 1023339.3286762601),"[44.9049109610093,-93.27422748323005]",4.467620,0 - 5409 2ND AVE S
13835,13836,0,3200,0,3298,STEVENS AVE,A40,None,None,55408,...,None,d0e6cdf2-4a45-4fed-ad52-d1d08987f6e8,1,1.0,284.579207,"LINESTRING (2812276.703 1037938.420, 2812288.7...",POINT (2812282.749692778 1037608.6316630788),"[44.94405920098535,-93.27538712586676]",2.451714,0 - 3298 STEVENS AVE
13836,13837,3501,3500,3599,3598,DUPONT AVE N,A40,None,None,55412,...,None,0e9545fe-1446-4dad-a6eb-852f386aa649,1,1.0,283.900843,"LINESTRING (2807469.624 1064722.569, 2807463.6...",POINT (2807466.6274603587 1065051.1877882767),"[45.0194420169815,-93.29307047393891]",4.923370,3501 - 3598 DUPONT AVE N


In [36]:
address_df.drop(address_df.columns.difference(['Display','geometry','latlon']), 1,inplace=True)
centerline_df.drop(centerline_df.columns.difference(['GBSID','LOCATION','geometry','centroid','latlon']), 1,inplace=True)

<ipython-input-36-7278371fcd71>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  address_df.drop(address_df.columns.difference(['Display','geometry','latlon']), 1,inplace=True)
<ipython-input-36-7278371fcd71>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  centerline_df.drop(centerline_df.columns.difference(['GBSID','LOCATION','geometry','centroid','latlon']), 1,inplace=True)


In [37]:
address_df

,Display,geometry,latlon
0,1 10TH AVE S,POINT (2818771.290 1050160.154),"[44.9783240770773,-93.2498835606176]"
1,1 17TH AVE N,POINT (2812057.097 1056015.745),"[44.9945503320143,-93.2756286253819]"
2,1 19TH AVE S,POINT (2820105.914 1049488.066),"[44.976447027974,-93.2447502008086]"
3,1 19TH ST E,POINT (2811687.670 1044779.946),"[44.9637426901606,-93.2774405102419]"
4,1 1ST ST N,POINT (2814191.323 1052004.738),"[44.9834971540659,-93.2675179891081]"
...,...,...,...
154701,116 1ST AVE N #B,POINT (2813836.311 1052285.938),"[44.9842771228172,-93.2688802608803]"
154702,2730 10TH AVE S,POINT (2816038.259 1040992.024),"[44.9532467352968,-93.2607651698611]"
154703,250 PORTLAND AVE #E,POINT (2815909.952 1049984.415),"[44.9779135621923,-93.2609467220749]"
154704,260 PORTLAND AVE #B,POINT (2815909.952 1049984.415),"[44.9779135621923,-93.2609467220749]"


In [38]:
centerline_df

,GBSID,geometry,centroid,latlon,LOCATION
0,12198,"LINESTRING (2824182.848 1033436.019, 2824482.7...",POINT (2824343.3433687184 1033437.1651246486),"[44.932303725970755,-93.22896590095331]",3000 - 3099 39TH ST E
1,20682,"LINESTRING (2802664.075 1052261.529, 2802650.1...",POINT (2802651.7366334866 1052286.376582747),"[44.9845465414285,-93.31209854486539]",601 - 612 SHERIDAN AVE N
2,22577,"LINESTRING (2804792.395 1021375.391, 2805124.6...",POINT (2804958.5136064384 1021375.8432505609),"[44.8997109417065,-93.30420711579976]",1998 - 1901 57TH ST W
3,11676,"LINESTRING (2814357.285 1032702.547, 2814683.8...",POINT (2814520.54720823 1032703.9520650019),"[44.930552156256596,-93.26691461978174]",600 - 619 40TH ST E
4,11050,"LINESTRING (2829825.169 1038146.805, 2829818.2...",POINT (2829821.695328156 1038461.1595069288),"[44.9459510802885,-93.207620164627]",3198 - 3101 46TH AVE S
...,...,...,...,...,...
13833,20733,"LINESTRING (2817010.288 1026439.590, 2817095.3...",POINT (2817174.151779763 1026491.6806034332),"[44.9134285378557,-93.2568797935352]",1200 - 1299 EAST MINNEHAHA PKWY
13834,26007,"LINESTRING (2812708.453 1023238.598, 2812715.9...",POINT (2812712.3810812924 1023339.3286762601),"[44.9049109610093,-93.27422748323005]",0 - 5409 2ND AVE S
13835,14082,"LINESTRING (2812276.703 1037938.420, 2812288.7...",POINT (2812282.749692778 1037608.6316630788),"[44.94405920098535,-93.27538712586676]",0 - 3298 STEVENS AVE
13836,10410,"LINESTRING (2807469.624 1064722.569, 2807463.6...",POINT (2807466.6274603587 1065051.1877882767),"[45.0194420169815,-93.29307047393891]",3501 - 3598 DUPONT AVE N


#Data Prep - Scooters

In [39]:
#!pip uninstall -q scikit-learn -y

In [40]:
!#pip install -q scikit-learn==0.23.2
#!pip install -q scikit-learn

In [41]:
#import sklearn
#sklearn.__version__

In [42]:
scoot_2019_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/scooter_data/Scooter_Availability_2019.csv',parse_dates=[0]).set_index('ESRI_OID')
scoot_2020_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/scooter_data/Scooter_Availability_2020.csv',parse_dates=[1]).set_index('ObjectId')
scoot_2021_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/scooter_data/Scooter_Availability_2021.csv',parse_dates=[1]).set_index('ObjectId')

In [43]:
neighborhoods_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/scooter_data/Minneapolis_Neighborhoods.csv')

In [44]:
neighborhoods_df.drop(neighborhoods_df.columns.difference(['OBJECTID','BDNAME']), 1,inplace=True)

<ipython-input-44-a9043fabfd21>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  neighborhoods_df.drop(neighborhoods_df.columns.difference(['OBJECTID','BDNAME']), 1,inplace=True)


In [45]:
neighborhoods_df

,OBJECTID,BDNAME
0,1,Phillips West
1,2,Downtown West
2,3,Downtown East
3,4,Ventura Village
4,5,Sumner - Glenwood
...,...,...
82,83,Marshall Terrace
83,84,Whittier
84,85,Lyndale
85,86,East Bde Maka Ska


In [46]:
scooter_df = pd.concat([scoot_2019_df,scoot_2020_df,scoot_2021_df])

In [47]:
scooter_df

,PollTime,CompanyName,NumberAvailable,ClosestCenterlineID,ClosestCenterlineType,Neighborhood
1,2019-05-16 19:00:00+00:00,Lime,1,10122.00,street,Webber - Camden
2,2019-05-16 19:00:00+00:00,Lime,1,10464.00,street,Lind - Bohanon
3,2019-05-16 19:00:00+00:00,Lime,1,10801.00,street,Audubon Park
4,2019-05-16 19:00:00+00:00,Lime,1,11395.00,street,CARAG
5,2019-05-16 19:00:00+00:00,Lime,1,13067.00,street,Minnehaha
...,...,...,...,...,...,...
795027,2021-12-15 12:59:59+00:00,Bird,1,18660.00,street,Near - North
795028,2021-12-15 12:59:59+00:00,Bird,1,14374.00,street,Sheridan
795029,2021-12-15 12:59:59+00:00,Bird,1,21851.00,street,University of Minnesota
795030,2021-12-15 12:59:59+00:00,Bird,1,09491384-4E68-4097-8AC5-01CF3ACBFC30,trail,Marcy Holmes


In [48]:
#Perform date conversions based on the poll time
scooter_df['month'] = [i.month for i in scooter_df['PollTime']]
scooter_df['year'] = [i.year for i in scooter_df['PollTime']]
scooter_df['day_of_week'] = [i.dayofweek for i in scooter_df['PollTime']]
scooter_df['day_of_year'] = [i.dayofyear for i in scooter_df['PollTime']]
scooter_df['hour'] = [i.hour for i in scooter_df['PollTime']]

In [49]:
#Drop trail types since data note available
scooter_df.drop(scooter_df[scooter_df.ClosestCenterlineType == "trail"].index, inplace=True)

In [50]:
#Create buckets for hours to reduce sparsity
scooter_df['hour'] = scooter_df['hour'].apply(lambda x: "12am-5am" if x >= 0 and x < 6 else ('6am-8am' if x >= 6 and x<9 else ('9am-11am' if x>=9 and x<12 else ('12pm-2pm' if x>=12 and x<15 else ('3pm-5pm' if x>=15 and x<18 else ('6pm-8pm' if x>=18 and x<21 else ('9pm-11pm')))))))


In [51]:
scooter_df.shape[0]

1319367

In [52]:
scooter_df.drop_duplicates(inplace=True)

In [53]:
scooter_df.shape[0]

1319367

In [54]:
#scooter_df = pd.merge(scooter_df, neighborhoods_df, how='left', left_on="Neighborhood", right_on="BDNAME")
scooter_df

,PollTime,CompanyName,NumberAvailable,ClosestCenterlineID,ClosestCenterlineType,Neighborhood,month,year,day_of_week,day_of_year,hour
1,2019-05-16 19:00:00+00:00,Lime,1,10122.00,street,Webber - Camden,5,2019,3,136,6pm-8pm
2,2019-05-16 19:00:00+00:00,Lime,1,10464.00,street,Lind - Bohanon,5,2019,3,136,6pm-8pm
3,2019-05-16 19:00:00+00:00,Lime,1,10801.00,street,Audubon Park,5,2019,3,136,6pm-8pm
4,2019-05-16 19:00:00+00:00,Lime,1,11395.00,street,CARAG,5,2019,3,136,6pm-8pm
6,2019-05-16 19:00:00+00:00,Lime,1,13621.00,street,Prospect Park - East River Road,5,2019,3,136,6pm-8pm
...,...,...,...,...,...,...,...,...,...,...,...
795026,2021-12-15 12:59:59+00:00,Bird,4,13713.00,street,Sheridan,12,2021,2,349,12pm-2pm
795027,2021-12-15 12:59:59+00:00,Bird,1,18660.00,street,Near - North,12,2021,2,349,12pm-2pm
795028,2021-12-15 12:59:59+00:00,Bird,1,14374.00,street,Sheridan,12,2021,2,349,12pm-2pm
795029,2021-12-15 12:59:59+00:00,Bird,1,21851.00,street,University of Minnesota,12,2021,2,349,12pm-2pm


In [55]:
def copy(df):
  return df.copy()

def drop(df, col):
    new_df = df.drop(col, axis=1)
    return new_df

def merge_neighbordhoods(df,n_df):
  new_df = df.copy()
  new_df = pd.merge(new_df, n_df, how='left', left_on="Neighborhood", right_on="BDNAME")
  return new_df

def ordinal_encode(df, col, clist):
  new_df = df.copy()
  new_df[col] = OrdinalEncoder(categories=[clist]).fit_transform(df[[col]])
  return new_df

def get_dummies(df, col):
    new_df = pd.get_dummies(df[col], prefix=col)
    new_df = pd.concat([df, new_df], axis = 1)
    new_df.drop(col, axis=1, inplace=True)
    return new_df

def num_convert(df,col):
  new_df = df.copy()
  new_df[col]=pd.to_numeric(df[col])
  return new_df

def update_names(df):
  new_df = df.copy()
  new_df.replace("ECCO","East Bde Maka Ska",inplace=True)
  new_df.replace("CARAG","South Uptown",inplace=True)
  new_df.replace("West Calhoun","West Maka Ska",inplace=True)
  return new_df

def rename_col(df,old,new):
  new_df = df.copy()
  new_df.rename(columns={old: new},inplace=True)
  return new_df

#helps with reducing memory usage, Optimizes variable sizing
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [56]:
def pipeline(df):
    return (df.pipe(copy)
            .pipe(update_names)
            .pipe(merge_neighbordhoods, neighborhoods_df)
            .pipe(drop, "PollTime")
            .pipe(drop, "Neighborhood")
            .pipe(drop, "BDNAME")
            .pipe(rename_col, "OBJECTID","Neighborhood")
            .pipe(drop, "ClosestCenterlineType")
            .pipe(ordinal_encode,"hour",['12am-5am', '6am-8am', '9am-11am', '12pm-2pm',
       '3pm-5pm', '6pm-8pm', '9pm-11pm'])
            .pipe(get_dummies, "CompanyName")
            .pipe(num_convert, "ClosestCenterlineID")
            .pipe(reduce_mem_usage,False))

In [57]:
clean_scooter_df = pipeline(scooter_df)

In [58]:
clean_scooter_df.shape[0]

1319367

In [59]:
clean_scooter_df.drop_duplicates(inplace=True)

In [60]:
clean_scooter_df.shape[0]

1130397

In [61]:
clean_scooter_df

,NumberAvailable,ClosestCenterlineID,month,year,day_of_week,day_of_year,hour,Neighborhood,CompanyName_Bird,CompanyName_Lime,CompanyName_Lyft,CompanyName_Spin
0,1,10120.0,5,2019,3,136,5.0,8,0,1,0,0
1,1,10464.0,5,2019,3,136,5.0,7,0,1,0,0
2,1,10800.0,5,2019,3,136,5.0,48,0,1,0,0
3,1,11392.0,5,2019,3,136,5.0,43,0,1,0,0
4,1,13624.0,5,2019,3,136,5.0,20,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1319362,4,13712.0,12,2021,2,349,3.0,51,1,0,0,0
1319363,1,18656.0,12,2021,2,349,3.0,25,1,0,0,0
1319364,1,14376.0,12,2021,2,349,3.0,51,1,0,0,0
1319365,1,21856.0,12,2021,2,349,3.0,21,1,0,0,0


In [62]:
clean_scooter_nn_df = clean_scooter_df.drop("Neighborhood", axis=1)

## Scoring on Full Set

In [63]:
X = clean_scooter_nn_df.drop("NumberAvailable", axis=1)
y = clean_scooter_nn_df["NumberAvailable"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=37)
print(X_train.shape,X_test.shape)

(904317, 10) (226080, 10)


In [64]:
knn_model = KNeighborsRegressor(n_neighbors=5, weights="distance")
knn_model.fit(X_train, y_train)
train_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

0.765285779888847

In [65]:
knn_model2 = KNeighborsRegressor(n_neighbors=3)
knn_model2.fit(X_train, y_train)
train_preds = knn_model2.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

1.0401698500769792

In [66]:
knn_model3 = KNeighborsRegressor(n_neighbors=7, weights="distance")
knn_model3.fit(X_train, y_train)
train_preds = knn_model3.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

0.7648445888668709

In [67]:
knn_model4 = KNeighborsRegressor(n_neighbors=3, weights="distance")
knn_model4.fit(X_train, y_train)
train_preds = knn_model4.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

0.7742252638852516

In [68]:
prediction_knn = knn_model.predict(
    pd.DataFrame([[14376, 12, 2022, 7, 343, 17, 1,1,1,1]], 
                 columns=["ClosestCenterlineID","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"]))
prediction_knn[0]

1.0

In [ ]:
rf = RandomForestRegressor(n_estimators=50, oob_score=True)
rf.fit(X_train, y_train)
train_preds = rf.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

In [ ]:
prediction_rf = rf.predict(
    pd.DataFrame([[14376, 12, 2022, 7, 343, 17, 1,1,1,1]], 
                 columns=["ClosestCenterlineID","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"]))
prediction_rf[0]

In [ ]:
rf2 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=True,
                      verbose=0, warm_start=False)

rf2.fit(X_train, y_train)
train_preds = rf2.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

In [ ]:
prediction_rf2 = rf2.predict(
    pd.DataFrame([[14376, 12, 2022, 7, 343, 17, 1,1,1,1]], 
                 columns=["ClosestCenterlineID","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"]))
prediction_rf2[0]

## Trying PyCaret

In [ ]:
#s = setup(clean_scooter_nn_df, target = 'NumberAvailable', fold = 3,
#          numeric_features = ["ClosestCenterlineID","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"])
    
    # compare all models and select best one based on RMSE
#best_model = compare_models(sort = 'RMSE', verbose=True)

#Pickle Output

In [ ]:
#filename = '/content/drive/MyDrive/Colab Notebooks/scooter_data/scooter_pickle.sav'
#pickle.dump({'model':knn_model,'addresses':address_df,'centerlines':centerline_df}, open(filename, 'wb'))


#Scoring & Mapping Together

In [70]:
import json
import numpy as np 
import pandas as pd
import os
import re

import datetime

from pandas.io.json import json_normalize
from matplotlib import pyplot as plt

from IPython.display import display
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor

import pickle

In [71]:
filename = '/content/drive/MyDrive/Colab Notebooks/scooter_data/scooter_pickle.sav'
with open(filename, 'rb') as fIn:
    scooter_pkl = pickle.load(fIn)

model = scooter_pkl['model']
address_df = scooter_pkl['addresses']
centerline_df = scooter_pkl['centerlines']

In [72]:
address = "3519 40TH AVE S" #@param {type:"string"}
coordinates = address_df.loc[address_df['Display'] == address]

NameError: ignored

In [73]:
start = address_df.sample()
start

,Display,geometry,latlon
90976,3519 40TH AVE S,POINT (2827952.878 1035925.881),"[44.9390469583024,-93.2149317819823]"


In [74]:
origin_coords = start.iloc[0]['geometry']
origin_latlong = start.iloc[0]['latlon']

In [75]:
def find_nearest(coords,df):
    distances = df.apply(lambda r: origin_coords.distance(r['centroid']), 
        axis=1)
    return df.loc[distances.idxmin(), 'latlon']

In [76]:
def find_within_dist(coords,df,dist):
    df['distances'] = df.apply(lambda r: origin_coords.distance(r['centroid'])  / 5279.98944, 
        axis=1)
    return df[(df['distances'] <= dist)]

In [77]:
def get_coordinates(address):
    #Return the coordinates associated with address
    coordinates = address_df.loc[address_df['Display'] == address]
    return coordinates

In [78]:
def make_prediction(centerline, month, year, day_of_week, day_of_year, hour, cn_bird, cn_lime, cn_lyft, cn_spin):
  entry = pd.DataFrame([[centerline, month, year, day_of_week, day_of_year, hour, cn_bird, cn_lime, cn_lyft, cn_spin]], 
                 columns=["ClosestCenterlineID","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"])
  prediction = model.predict(entry)
  return prediction[0]

In [79]:
dest_latlong = find_nearest(origin_coords,centerline_df)
dest_latlong

'[44.93874584912055,-93.21530427404795]'

In [80]:
within_df = find_within_dist(origin_coords,centerline_df,0.1)
within_df

,GBSID,geometry,centroid,latlon,LOCATION,distances
2547,12251,"LINESTRING (2828189.627 1035492.921, 2828185.5...",POINT (2828187.5923021063 1035823.1142499843),"[44.9387589108638,-93.2140292126605]",3598 - 3501 41ST AVE S,0.048528
4252,11202,"LINESTRING (2828185.557 1036153.307, 2828516.5...",POINT (2828351.0645418838 1036156.5026202465),"[44.93966899108595,-93.21338555425186]",4100 - 4199 35TH ST E,0.087150
4603,12250,"LINESTRING (2827859.224 1035487.227, 2828189.6...",POINT (2828024.4257045486 1035490.0739350542),"[44.9378497700646,-93.2146716577697]",4000 - 4099 36TH ST E,0.083644
5873,12249,"LINESTRING (2827859.224 1035487.227, 2827855.7...",POINT (2827857.481089263 1035815.1803397711),"[44.93874584912055,-93.21530427404795]",3598 - 3501 40TH AVE S,0.027677
7792,11210,"LINESTRING (2827855.738 1036143.134, 2828185.5...",POINT (2828020.6476868214 1036148.2206547011),"[44.939654989919745,-93.21466182893874]",4000 - 4099 35TH ST E,0.044023
8725,11211,"LINESTRING (2827525.351 1036138.598, 2827855.7...",POINT (2827690.544699679 1036140.8659776596),"[44.93964350962915,-93.2159368567429]",3900 - 3999 35TH ST E,0.064237
10545,12247,"LINESTRING (2827530.186 1035479.576, 2827525.3...",POINT (2827527.7687296793 1035809.0868643839),"[44.938737810584854,-93.2165777265769]",3598 - 3501 39TH AVE S,0.083497
10580,12248,"LINESTRING (2827530.186 1035479.576, 2827859.2...",POINT (2827694.705119264 1035483.4012264952),"[44.937840150076255,-93.21594514388195]",3900 - 3999 36TH ST E,0.097025


In [81]:
month = 12
year = 2022
day_of_week = 2
day_of_year = 345
hour = 17
cn_bird = 0
cn_lime = 0
cn_lyft = 1
cn_spin = 0


In [82]:
m = folium.Map(location=json.loads(origin_latlong), zoom_start=18)
folium.Marker(json.loads(origin_latlong)).add_to(m)
#folium.Marker(json.loads(dest_latlong),icon=folium.Icon(color='red')).add_to(m)
for _, r in within_df.iterrows():
  centerline = r['GBSID']
  scooters = round(make_prediction(centerline, month, year, day_of_week, day_of_year, hour, cn_bird, cn_lime, cn_lyft, cn_spin),0)
  folium.Marker(json.loads(r['latlon']),popup="<i> Expected Available: " + str(scooters) + "</i>",icon=folium.Icon(color='green')).add_to(m)
m

#Building Negative Cases

In [83]:
 full_negatives_df = pd.DataFrame(columns=["NumberAvailable","ClosestCenterlineID","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"])

In [84]:

 negatives_df = pd.DataFrame(columns=["NumberAvailable","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"])

In [85]:
#hours_2019 = [3,7,11,15,19,23]
#hours_2020 = [0,3,5,8,9,12,15,17,20,21]
#hours_2021 = [0,3,5,8,9,12,15,16,17,20,21]

hours = ['12am-5am','6am-8am','9am-11am','12pm-2pm','3pm-5pm','6pm-8pm','9pm-11pm']

In [86]:
for y in range(2019, 2022):
  for dy in range(1, 367) if y == 2020 else range(1,366):
    n_date = datetime.datetime(y, 1, 1) + datetime.timedelta(dy - 1)
    m = n_date.month
    dw = n_date.weekday()
    for h in hours:
      record = pd.DataFrame([[0, m, y, dw, dy, h, 0, 0, 0, 0]], 
                columns=["NumberAvailable","month","year","day_of_week","day_of_year","hour","CompanyName_Bird","CompanyName_Lime","CompanyName_Lyft","CompanyName_Spin"])
      negatives_df = negatives_df.append(record)

In [87]:
negatives_df

,NumberAvailable,month,year,day_of_week,day_of_year,hour,CompanyName_Bird,CompanyName_Lime,CompanyName_Lyft,CompanyName_Spin
0,0,1,2019,1,1,12am-5am,0,0,0,0
0,0,1,2019,1,1,6am-8am,0,0,0,0
0,0,1,2019,1,1,9am-11am,0,0,0,0
0,0,1,2019,1,1,12pm-2pm,0,0,0,0
0,0,1,2019,1,1,3pm-5pm,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
0,0,12,2021,4,365,9am-11am,0,0,0,0
0,0,12,2021,4,365,12pm-2pm,0,0,0,0
0,0,12,2021,4,365,3pm-5pm,0,0,0,0
0,0,12,2021,4,365,6pm-8pm,0,0,0,0


In [88]:
centerlines_2019 = pd.DataFrame(clean_scooter_df[clean_scooter_df['year']==2019]['ClosestCenterlineID'].unique())
centerlines_2020 = pd.DataFrame(clean_scooter_df[clean_scooter_df['year']==2020]['ClosestCenterlineID'].unique())
centerlines_2021 = pd.DataFrame(clean_scooter_df[clean_scooter_df['year']==2021]['ClosestCenterlineID'].unique())
scooter_centerlines_df = pd.DataFrame(clean_scooter_df['ClosestCenterlineID'].unique())

In [89]:
scooter_centerlines_df

,0
0,10120.0
1,10464.0
2,10800.0
3,11392.0
4,13624.0
...,...
1285,13568.0
1286,15776.0
1287,23504.0
1288,15688.0


In [90]:
for _, r in scooter_centerlines_df.iterrows():
  df = negatives_df
  df["ClosestCenterlineID"] = r[0]
  full_negatives_df = full_negatives_df.append(df)

In [91]:
full_negatives_df

,NumberAvailable,ClosestCenterlineID,month,year,day_of_week,day_of_year,hour,CompanyName_Bird,CompanyName_Lime,CompanyName_Lyft,CompanyName_Spin
0,0,10120.0,1,2019,1,1,12am-5am,0,0,0,0
0,0,10120.0,1,2019,1,1,6am-8am,0,0,0,0
0,0,10120.0,1,2019,1,1,9am-11am,0,0,0,0
0,0,10120.0,1,2019,1,1,12pm-2pm,0,0,0,0
0,0,10120.0,1,2019,1,1,3pm-5pm,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
0,0,14696.0,12,2021,4,365,9am-11am,0,0,0,0
0,0,14696.0,12,2021,4,365,12pm-2pm,0,0,0,0
0,0,14696.0,12,2021,4,365,3pm-5pm,0,0,0,0
0,0,14696.0,12,2021,4,365,6pm-8pm,0,0,0,0


In [93]:
full_negatives_df.to_csv('/content/drive/MyDrive/Colab Notebooks/scooter_data/negative_cases.csv')